# Segmenting and Clustering Neighborhoods in Toronto (Part II)

## Loading Canada Borough data into Pandas

In [2]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# loading borough data of Canada into pandas dataframe
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df = df[df.Borough!='Not assigned']

# loading lat and long values into another pandas dataframe
lat_long = pd.read_csv('http://cocl.us/Geospatial_data')

# merging both dataframes
can = df.merge(lat_long,left_on = 'Postal code',right_on = 'Postal Code')[['Postal code','Borough','Neighborhood','Latitude','Longitude']]
can.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [9]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

In [10]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

## Importing Required Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

print('All Libraries imported.')

ModuleNotFoundError: No module named 'folium'

In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(can['Borough'].unique()),
        can.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [11]:
Toronto_data = can[can['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [15]:
# Getting Totonto lat and long values.
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [1]:
CLIENT_ID = '' 
CLIENT_SECRET = '' 
VERSION = '20200427' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [18]:
Toronto_data.loc[0, 'Neighborhood']

'Regent Park / Harbourfront'

## Using function to extract venue data

In [21]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:

Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Roseda

In [23]:
Toronto_venues.shape

(1623, 7)

In [24]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [133]:
#Toronto_venues.groupby('Venue').count()

In [36]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].value_counts())))

There are 226 uniques categories.


In [33]:
len(Toronto_venues['Venue Category'].value_counts())

226

In [80]:
# one hot encoding
Toronto_hot = pd.get_dummies(Toronto_venues[['Venue Category']],prefix="", prefix_sep="")
Toronto_hot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [81]:
Toronto_hot['Neighborhood'] = Toronto_venues['Neighborhood']

In [82]:
fixed_columns = ['Neighborhood']+list(set(Toronto_hot.columns.values.tolist()).difference(set(['Neighborhood'])))

In [83]:
Toronto_hot = Toronto_hot[fixed_columns]

## Grouping the hot encoded data by neighborhood.

In [88]:
Toronto_hot_grouped = Toronto_hot.groupby('Neighborhood').mean().reset_index()

In [89]:
Toronto_hot_grouped.head()

,Neighborhood,Discount Store,Flea Market,Sporting Goods Shop,Restaurant,Bookstore,Auto Workshop,Airport Lounge,Farmers Market,Airport Terminal,Latin American Restaurant,Wine Bar,Distribution Center,Dessert Shop,Fruit & Vegetable Store,Gym / Fitness Center,Lounge,Boutique,Grocery Store,Ice Cream Shop,Spa,Frozen Yogurt Shop,Market,Burrito Place,Lake,Lingerie Store,Garden,Roof Deck,Boat or Ferry,Baseball Stadium,Basketball Stadium,Tailor Shop,Ramen Restaurant,Art Gallery,Bistro,Gastropub,Concert Hall,Irish Pub,Airport Food Court,Record Shop,Theme Restaurant,Stadium,Noodle House,Speakeasy,Poutine Place,Bed & Breakfast,Café,College Rec Center,Miscellaneous Shop,Donut Shop,Eastern European Restaurant,Cajun / Creole Restaurant,Smoke Shop,Sake Bar,Japanese Restaurant,Chinese Restaurant,Dog Run,Vietnamese Restaurant,Brazilian Restaurant,General Entertainment,Belgian Restaurant,Movie Theater,Sculpture Garden,Bus Line,Cheese Shop,College Auditorium,Italian Restaurant,Toy / Game Store,Monument / Landmark,Gluten-free Restaurant,Beach,Cupcake Shop,Doner Restaurant,Mediterranean Restaurant,Church,Thai Restaurant,French Restaurant,Deli / Bodega,Convenience Store,Fried Chicken Joint,Office,Furniture / Home Store,Comic Shop,Plaza,Park,Taiwanese Restaurant,Beer Store,American Restaurant,Food Court,Gaming Cafe,Building,Convention Center,Seafood Restaurant,Breakfast Spot,Filipino Restaurant,Food Truck,Pharmacy,Harbor / Marina,IT Services,Men's Store,Event Space,Garden Center,Bar,Shoe Store,Gay Bar,Music Venue,Skate Park,Creperie,Asian Restaurant,Electronics Store,Food & Drink Shop,Hotel Bar,Intersection,Gym,College Gym,Art Museum,Scenic Lookout,Airport,Women's Store,Health Food Store,Juice Bar,Martial Arts Dojo,Video Game Store,Sandwich Place,Chocolate Shop,Hospital,Molecular Gastronomy Restaurant,General Travel,Bagel Shop,Sushi Restaurant,Comfort Food Restaurant,Tanning Salon,Tea Room,Korean Restaurant,Mexican Restaurant,Falafel Restaurant,Middle Eastern Restaurant,Rental Car Location,Steakhouse,Baby Store,Dumpling Restaurant,Butcher,Vegetarian / Vegan Restaurant,Gourmet Shop,Pet Store,Performing Arts Venue,Swim School,Gift Shop,Candy Store,Opera House,Caribbean Restaurant,Ethiopian Restaurant,Skating Rink,Burger Joint,Antique Shop,Cocktail Bar,Bakery,Department Store,Cable Car,Dance Studio,Arts & Crafts Store,Beer Bar,Clothing Store,Coworking Space,New American Restaurant,Salad Place,Trail,Snack Place,History Museum,Strip Club,Theater,Fish & Chips Shop,Sports Bar,Train Station,Museum,Portuguese Restaurant,Modern European Restaurant,Indie Movie Theater,Hotel,Jazz Club,Flower Shop,Poke Place,Moroccan Restaurant,Fish Market,Cosmetics Shop,Coffee Shop,Fast Food Restaurant,Stationery Store,Liquor Store,Shopping Mall,Board Shop,Climbing Gym,German Restaurant,Other Great Outdoors,Yoga Studio,Greek Restaurant,Airport Service,Historic Site,Indian Restaurant,Pizza Place,Playground,Hobby Shop,Gas Station,College Arts Building,Airport Gate,Nightclub,Brewery,Bubble Tea Shop,Light Rail Station,Food,Soup Place,Salon / Barbershop,Hookah Bar,Supermarket,Organic Grocery,Optical Shop,Bank,Diner,BBQ Joint,Colombian Restaurant,Fountain,Aquarium,Pub,Cuban Restaurant,Health & Beauty Service,Jewelry Store
0,Berczy Park,0.000000,0.0,0.017241,0.034483,0.0,0.000000,0.000,0.034483,0.000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.017241,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.017241,0.017241,0.000000,0.017241,0.017241,0.000000,0.017241,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,0.000000,0.017241,0.0,0.000000,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.034483,0.0,0.034483,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.017241,0.017241,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.017241,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0

In [109]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Getting the top 10 venue's for each Neighborhood

In [110]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_hot_grouped['Neighborhood']

for ind in np.arange(Toronto_hot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_hot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Café,Farmers Market,Italian Restaurant,Bakery,Beer Bar,Restaurant
1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Nightclub,Breakfast Spot,Gym,Italian Restaurant,Climbing Gym,Furniture / Home Store,Pet Store,Performing Arts Venue
2,Business reply mail Processing CentrE,Light Rail Station,Smoke Shop,Garden Center,Fast Food Restaurant,Comic Shop,Park,Garden,Pizza Place,Burrito Place,Spa
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Coffee Shop,Airport Food Court,Airport Gate,Harbor / Marina,Bar,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Fried Chicken Joint,Ice Cream Shop,Burger Joint,Bubble Tea Shop,Gastropub


In [130]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_hot_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

## Loading Cluster data into dataframe.

In [114]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [117]:
Toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


## Creating final dataframe for our project.

In [121]:
final_ds = Toronto_data.drop('Postal code',1).merge(neighborhoods_venues_sorted, left_on='Neighborhood',right_on='Neighborhood')

## Displaying the segmented clusters on Toronto map using folium.

In [123]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_ds['Latitude'], final_ds['Longitude'], final_ds['Neighborhood'], final_ds['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [124]:
final_ds.loc[final_ds['Cluster Labels'] == 0, final_ds.columns[[1] + list(range(5, final_ds.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regent Park / Harbourfront,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Restaurant,Theater,Café,Playground,Historic Site
1,Queen's Park / Ontario Provincial Government,Coffee Shop,Sushi Restaurant,Diner,Bar,Theater,Beer Bar,Arts & Crafts Store,Burger Joint,Vegetarian / Vegan Restaurant,Mexican Restaurant
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Bookstore,Theater
3,St. James Town,Coffee Shop,Café,Italian Restaurant,Cocktail Bar,Gastropub,American Restaurant,Creperie,Department Store,Clothing Store,Theater
4,The Beaches,Trail,Pub,Health Food Store,Park,General Entertainment,Belgian Restaurant,Comic Shop,Furniture / Home Store,Office,Fried Chicken Joint
5,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Café,Farmers Market,Italian Restaurant,Bakery,Beer Bar,Restaurant
6,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Fried Chicken Joint,Ice Cream Shop,Burger Joint,Bubble Tea Shop,Gastropub
7,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Gas Station,Candy Store,Nightclub,Baby Store,Restaurant
8,Richmond / Adelaide / King,Coffee Shop,Café,Restaurant,Gym,American Restaurant,Deli / Bodega,Thai Restaurant,Hotel,Clothing Store,Seafood Restaurant
9,Dufferin / Dovercourt Village,Pharmacy,Bakery,Gym / Fitness Center,Bar,Brewery,Café,Supermarket,Bank,Grocery Store,Smoke Shop


## Cluster 2

In [125]:
final_ds.loc[final_ds['Cluster Labels'] == 1, final_ds.columns[[1] + list(range(5, final_ds.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Moore Park / Summerhill East,Playground,Jewelry Store,Cupcake Shop,Comic Shop,Furniture / Home Store,Office,Fried Chicken Joint,Convenience Store,Deli / Bodega,French Restaurant


## Cluster 3

In [126]:
final_ds.loc[final_ds['Cluster Labels'] == 2, final_ds.columns[[1] + list(range(5, final_ds.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Garden,Jewelry Store,Park,Comic Shop,Furniture / Home Store,Office,Fried Chicken Joint,Convenience Store,Deli / Bodega,French Restaurant


## Cluster 4

In [131]:
final_ds.loc[final_ds['Cluster Labels'] == 3, final_ds.columns[[1] + list(range(5, final_ds.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Cupcake Shop,Furniture / Home Store,Office,Fried Chicken Joint,Convenience Store,Deli / Bodega


## Cluster 5

In [1]:
final_ds.loc[final_ds['Cluster Labels'] == 4, final_ds.columns[[1] + list(range(5, final_ds.shape[1]))]]

NameError: name 'final_ds' is not defined